In [6]:
import draft_mainmodule as mainmod
import load_and_save_data as load

{'run_ID': 'example_default', 'folder': 'example_default', 'timestep': 100000000000.0, 'r_planet': 250000.0, 'core_size_factor': 0.5, 'reg_fraction': 0.032, 'max_time': 400, 'temp_core_melting': 1200.0, 'olivine_cp': 819.0, 'olivine_density': 3341.0, 'cmb_conductivity': 3.0, 'core_cp': 850.0, 'core_density': 7800.0, 'temp_init': 1600.0, 'temp_surface': 250.0, 'core_temp_init': 1600.0, 'core_latent_heat': 270000.0, 'kappa_reg': 5e-08, 'dr': 1000.0, 'reg_percent': 'y', 'cond_constant': 'y', 'density_constant': 'y', 'heat_cap_constant': 'y'}


In [7]:
data = load.load_params_from_file()

In [8]:
(r_core,
radii,
core_radii,
reg_thickness,
where_regolith,
times,
mantle_temperature_array,
core_temperature_array) = mainmod.set_up()